# Feature Engineering v3 - GUIDE Dataset

**Obiettivo:** Feature engineering avanzato con target binario e encoding ottimizzato.

**Differenze rispetto a v2:**
- Target binario: BinaryIncidentGrade (1=TruePositive, 0=FalsePositive/BenignPositive)
- SmoothedRisk per AlertTitle (Bayesian smoothing)
- GeoLoc_freq invece di encoding geografico standard
- Frequency encoding per colonne ad alta cardinalità
- MITRE top 30 tecniche
- One-hot encoding selettivo solo per SuspicionLevel e EvidenceRole

**Pipeline:**
1. Caricamento e pulizia
2. Target binario
3. SmoothedRisk per AlertTitle
4. GeoLoc_freq
5. Features temporali
6. Frequency encoding categorie
7. One-hot encoding selettivo
8. Processing MITRE (top 30)
9. Aggregazione Evidence → Incident
10. Train/Test split e salvataggio

## 1. Setup

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import pickle
import os
import warnings
warnings.filterwarnings('ignore')

print("Librerie importate con successo!")

Librerie importate con successo!


## 2. Caricamento e Pulizia

In [4]:
print("Caricamento dataset...")
df = pd.read_csv('../data/GUIDE_Train.csv')

print(f"Dataset caricato: {df.shape[0]:,} righe, {df.shape[1]} colonne")

# Rimuovi record senza target
df = df[df['IncidentGrade'].notna()].copy()

# Rimuovi duplicati
#df = df.drop_duplicates(subset=['Id'], keep='first')

print(f"Dimensioni dopo pulizia: {df.shape}")

Caricamento dataset...
Dataset caricato: 9,516,837 righe, 45 colonne
Dimensioni dopo pulizia: (9465497, 45)


## 3. Target Binario

In [5]:
# Crea target binario: 1 = TruePositive, 0 = FalsePositive/BenignPositive
df['BinaryIncidentGrade'] = df['IncidentGrade'].apply(
    lambda x: 1 if x == 'TruePositive' else 0
)

counts = df['BinaryIncidentGrade'].value_counts()
percentages = df['BinaryIncidentGrade'].value_counts(normalize=True).mul(100).round(2)

result = pd.DataFrame({'Count': counts, 'Percentage': percentages})
print("\nDistribuzione Target Binario:")
print(result)
print(f"\nClass imbalance ratio: {counts[0]/counts[1]:.2f}:1")


Distribuzione Target Binario:
                       Count  Percentage
BinaryIncidentGrade                     
0                    6142784        64.9
1                    3322713        35.1

Class imbalance ratio: 1.85:1


## 4. SmoothedRisk per AlertTitle

**Wilson/Bayes Smoothing:** Corregge la media quando abbiamo pochi esempi.  
Con solo 2 esempi e 100% risk, lo smoothing porta il valore verso 0.5 per riflettere l'incertezza.

In [6]:
# Calcola risk grezzo per AlertTitle
alert_risk = df.groupby('AlertTitle')['BinaryIncidentGrade'].mean()
alert_count = df.groupby('AlertTitle')['BinaryIncidentGrade'].sum()

alert_summary = pd.DataFrame({
    'Risk': alert_risk,
    'Count': alert_count
})

# Bayesian smoothing
alpha = 2
beta = 2

alert_summary['SmoothedRisk'] = (
    alert_summary['Risk'] * alert_summary['Count'] + alpha
) / (alert_summary['Count'] + alpha + beta)

# Merge nel dataframe
df = df.merge(
    alert_summary[['SmoothedRisk']], 
    left_on='AlertTitle', 
    right_index=True, 
    how='left'
)

print(f"SmoothedRisk creato per {len(alert_summary)} AlertTitle univoci")
print(f"\nStatistiche SmoothedRisk:")
print(df['SmoothedRisk'].describe())

SmoothedRisk creato per 79952 AlertTitle univoci

Statistiche SmoothedRisk:
count    9.465497e+06
mean     4.901028e-01
std      2.674729e-01
min      1.622586e-02
25%      2.835467e-01
50%      5.000000e-01
75%      7.540790e-01
max      9.999631e-01
Name: SmoothedRisk, dtype: float64


## 5. GeoLoc_freq

Frequenza normalizzata della combinazione `CountryCode_State_City`.  
Utile per identificare location rare o pattern regionali.

In [7]:
# Crea identificatore geografico
df['GeoLoc'] = (
    df['CountryCode'].astype(str) + "_" + 
    df['State'].astype(str) + "_" + 
    df['City'].astype(str)
)

# Calcola frequenza normalizzata
geo_freq = df['GeoLoc'].value_counts(normalize=True)
df['GeoLoc_freq'] = df['GeoLoc'].map(geo_freq)

# Drop colonne geografiche originali
df.drop(columns=['CountryCode', 'State', 'City', 'GeoLoc'], inplace=True)

print("GeoLoc_freq creato")
print(df['GeoLoc_freq'].describe())

GeoLoc_freq creato
count    9.465497e+06
mean     8.479237e-01
std      2.481550e-01
min      1.056469e-07
25%      9.207343e-01
50%      9.207343e-01
75%      9.207343e-01
max      9.207343e-01
Name: GeoLoc_freq, dtype: float64


## 6. Features Temporali

In [8]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['month'] = df['Timestamp'].dt.month
df['hour'] = df['Timestamp'].dt.hour
df['weekday'] = df['Timestamp'].dt.weekday + 1
df['IsWeekend'] = (df['Timestamp'].dt.dayofweek >= 5).astype(int)

# Conserva Timestamp per aggregazione, poi rimuoveremo
print("Features temporali create: month, hour, weekday, IsWeekend")

Features temporali create: month, hour, weekday, IsWeekend


## 7. Gestione Missing e Frequency Encoding

**Frequency encoding** per colonne ad alta cardinalità evita curse of dimensionality.  
Fornisce un segnale semplice e limitato.

In [10]:
# Fill missing values
df['Roles'] = df['Roles'].fillna('missing')
df['ActionGrouped'] = df['ActionGrouped'].fillna('Missing')
df['SuspicionLevel'] = df['SuspicionLevel'].fillna('Missing')
df['LastVerdict'] = df['LastVerdict'].fillna('Missing')

# Group rare verdicts (< 100 occorrenze)
verdict_counts = df['LastVerdict'].value_counts()
rare_verdicts = verdict_counts[verdict_counts < 100].index
df['LastVerdict'] = df['LastVerdict'].replace(rare_verdicts, 'Other')

print("Missing values gestiti")

Missing values gestiti


In [11]:
# Frequency encoding per colonne ad alta cardinalità
freq_encode_cols = [
    'ThreatFamily', 'AntispamDirection', 'ActionGranular',
    'LastVerdict', 'ResourceType', 'Roles', 'ActionGrouped', 
    'EntityType', 'Category'
]

for col in freq_encode_cols:
    if col in df.columns:
        # Fill missing
        df[col] = df[col].fillna('Missing')
        
        # Frequency encode
        freq = df[col].value_counts(normalize=True)
        df[f"{col}_freq"] = df[col].map(freq)
        
        # Drop original
        df.drop(columns=col, inplace=True)
        print(f"  {col} -> {col}_freq")

print(f"\nFrequency encoding completato per {len(freq_encode_cols)} colonne")

  ThreatFamily -> ThreatFamily_freq
  AntispamDirection -> AntispamDirection_freq
  ActionGranular -> ActionGranular_freq
  LastVerdict -> LastVerdict_freq
  ResourceType -> ResourceType_freq
  Roles -> Roles_freq
  ActionGrouped -> ActionGrouped_freq
  EntityType -> EntityType_freq
  Category -> Category_freq

Frequency encoding completato per 9 colonne


## 8. One-Hot Encoding Selettivo

Solo per `SuspicionLevel` e `EvidenceRole` (bassa cardinalità e forte segnale).

In [14]:
onehot_cols = ['SuspicionLevel', 'EvidenceRole']

for col in onehot_cols:
    if col in df.columns:
        # Fill missing
        df[col] = df[col].fillna('Missing')
        
        # Group rare categories (< 100 occorrenze)
        counts = df[col].value_counts()
        rare = counts[counts < 100].index
        df[col] = df[col].replace(rare, 'Other')
        
        # One-hot encode (drop_first per evitare multicollinearità)
        df = pd.get_dummies(df, columns=[col], drop_first=True)
        print(f"  {col} -> one-hot encoded")

print("\nOne-hot encoding completato")


One-hot encoding completato


## 9. Processing MITRE Techniques (Top 30)

In [15]:
# Step 1: Split semicolon-separated string
df['MitreList'] = df['MitreTechniques'].apply(
    lambda x: x.split(';') if pd.notna(x) else []
)

# Step 2: Identifica top 30 tecniche
all_techs = [tech for sublist in df['MitreList'] for tech in sublist]
top_techs = [tech for tech, _ in Counter(all_techs).most_common(30)]
top_tech_set = set(top_techs)

print(f"Top 30 MITRE techniques selezionate")
print(f"Top 10: {Counter(all_techs).most_common(10)}")

Top 30 MITRE techniques selezionate
Top 10: [('T1078', 1467176), ('T1078.004', 1357022), ('T1566.002', 815149), ('T1566', 659978), ('T1110', 189348), ('T1133', 177415), ('T1566.001', 140374), ('T1110.003', 108831), ('T1110.001', 107510), ('T1071', 96343)]


In [16]:
# Step 3: Filtra ogni lista per includere solo top techniques
df['FilteredMitreList'] = df['MitreList'].apply(
    lambda x: [tech for tech in x if tech in top_tech_set]
)

# Step 4: One-hot encode con MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=top_techs)
tech_matrix = pd.DataFrame(
    mlb.fit_transform(df['FilteredMitreList']),
    columns=mlb.classes_, 
    index=df.index
)

# Step 5: Merge e drop colonne originali
df = pd.concat([df, tech_matrix], axis=1)
df.drop(columns=['MitreTechniques', 'MitreList', 'FilteredMitreList'], inplace=True)

print(f"\nMITRE features create: {tech_matrix.shape[1]}")
print(f"Shape dataset: {df.shape}")


MITRE features create: 30
Shape dataset: (9465497, 79)


## 10. Aggregazione a Livello Incident

In [17]:
def get_mode(x):
    mode = x.mode()
    return mode[0] if len(mode) > 0 else x.iloc[0] if len(x) > 0 else None

# Prepara aggregazioni
agg_dict = {
    'BinaryIncidentGrade': 'first',
    'IncidentGrade': 'first',
    'AlertId': 'nunique',
    'Id': 'count',
    'SmoothedRisk': 'mean',
    'GeoLoc_freq': 'mean',
    'hour': ['min', 'max', 'mean'],
    'month': get_mode,
    'weekday': get_mode,
    'IsWeekend': 'max',
    'Timestamp': ['min', 'max'],
}

# Aggiungi frequency-encoded columns (media)
freq_cols = [col for col in df.columns if col.endswith('_freq') and col not in ['GeoLoc_freq']]
for col in freq_cols:
    agg_dict[col] = 'mean'

# Aggiungi one-hot encoded columns (somma)
onehot_cols_created = [col for col in df.columns if col.startswith(('SuspicionLevel_', 'EvidenceRole_'))]
for col in onehot_cols_created:
    agg_dict[col] = 'sum'

# Aggiungi MITRE columns (somma)
mitre_cols = [col for col in df.columns if col.startswith('T') and len(col) <= 6]
for col in mitre_cols:
    agg_dict[col] = 'sum'

print(f"Aggregazioni preparate per {len(agg_dict)} features")

Aggregazioni preparate per 43 features


In [18]:
# Esegui aggregazione
incident_agg = df.groupby('IncidentId').agg(agg_dict).reset_index()

# Flatten colonne multi-livello
incident_agg.columns = [
    '_'.join(col).strip('_') if isinstance(col, tuple) else col 
    for col in incident_agg.columns.values
]

print(f"Dataset aggregato: {incident_agg.shape}")

Dataset aggregato: (448901, 47)


In [19]:
# Calcola durata e rinomina colonne
incident_agg['Duration_seconds'] = (
    pd.to_datetime(incident_agg['Timestamp_max']) - 
    pd.to_datetime(incident_agg['Timestamp_min'])
).dt.total_seconds()

rename_map = {
    'AlertId_nunique': 'NumAlerts',
    'Id_count': 'NumEvidences',
    'SmoothedRisk_mean': 'SmoothedRisk_avg',
    'GeoLoc_freq_mean': 'GeoLoc_freq_avg',
    'hour_min': 'Hour_First',
    'hour_max': 'Hour_Last',
    'hour_mean': 'Hour_Avg',
    'BinaryIncidentGrade_first': 'BinaryIncidentGrade',
    'IncidentGrade_first': 'IncidentGrade',
}

incident_agg = incident_agg.rename(columns=rename_map)
incident_agg = incident_agg.drop(columns=['Timestamp_min', 'Timestamp_max'], errors='ignore')

print(f"Features finali: {incident_agg.shape[1] - 3}")  # -3 per ID e 2 target

Features finali: 43


## 11. Preparazione per Modeling

In [20]:
# Separa features e target
X = incident_agg.drop(columns=['IncidentId', 'BinaryIncidentGrade', 'IncidentGrade'])
y = incident_agg['BinaryIncidentGrade']

# Gestisci missing (se presenti)
X = X.fillna(-999)

print(f"Features finali: {X.shape}")
print(f"Target: {y.shape}")
print(f"\nDistribuzione target binario:")
print(y.value_counts(normalize=True))

Features finali: (448901, 43)
Target: (448901,)

Distribuzione target binario:
BinaryIncidentGrade
0    0.787009
1    0.212991
Name: proportion, dtype: float64


## 12. Train/Test Split Stratificato

In [21]:
# Split stratificato (70/30)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"\nDistribuzione y_train:")
print(y_train.value_counts(normalize=True))
print(f"\nDistribuzione y_test:")
print(y_test.value_counts(normalize=True))

X_train: (314230, 43)
X_test: (134671, 43)

Distribuzione y_train:
BinaryIncidentGrade
0    0.78701
1    0.21299
Name: proportion, dtype: float64

Distribuzione y_test:
BinaryIncidentGrade
0    0.787007
1    0.212993
Name: proportion, dtype: float64


## 13. Salvataggio Dataset Processati

In [22]:
os.makedirs('../data/processed_v3', exist_ok=True)

X_train.to_csv('../data/processed_v3/X_train.csv', index=False)
X_test.to_csv('../data/processed_v3/X_test.csv', index=False)
y_train.to_csv('../data/processed_v3/y_train.csv', index=False, header=['BinaryIncidentGrade'])
y_test.to_csv('../data/processed_v3/y_test.csv', index=False, header=['BinaryIncidentGrade'])

# Salva anche il dataframe completo aggregato
incident_agg.to_csv('../data/processed_v3/incident_features.csv', index=False)

print("Dataset salvati in ../data/processed_v3/")
print(f"  - X_train.csv: {X_train.shape}")
print(f"  - X_test.csv: {X_test.shape}")
print(f"  - y_train.csv: {y_train.shape}")
print(f"  - y_test.csv: {y_test.shape}")
print(f"  - incident_features.csv: {incident_agg.shape}")
print(f"\nFeatures totali: {X_train.shape[1]}")
print(f"Target binario: 0 (FP/BP) vs 1 (TP)")

Dataset salvati in ../data/processed_v3/
  - X_train.csv: (314230, 43)
  - X_test.csv: (134671, 43)
  - y_train.csv: (314230,)
  - y_test.csv: (134671,)
  - incident_features.csv: (448901, 46)

Features totali: 43
Target binario: 0 (FP/BP) vs 1 (TP)


## Summary

**Features create:**
- Target binario (BinaryIncidentGrade)
- SmoothedRisk (Bayesian smoothing per AlertTitle)
- GeoLoc_freq (frequenza location)
- Temporal features (hour, month, weekday, IsWeekend, Duration_seconds)
- Frequency encoding per 9 colonne ad alta cardinalità
- One-hot encoding per SuspicionLevel e EvidenceRole
- MITRE top 30 tecniche (one-hot encoded)
- Aggregazioni a livello incident (count, mean, sum, mode)

**Pronto per training XGBoost con target binario!**